# 🧪 PyTorch Lab 2: Autoencoder 


## 0) Setup


In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
print('PyTorch version:', torch.__version__)

device = torch.device("cpu")

PyTorch version: 2.8.0+cpu


## 1) Data: load Fashion-MNIST, visualize, create Dataloader and add device
**Exercise 1** — Follow the previous lab to load format data and add the device

In [ ]:
# Charger le dataset d'entraînement avec la transformation ToTensor()
train_data = datasets.FashionMNIST(
    root='data/',       # dossier local pour stocker les données
    train=True,         # jeu d'entraînement
    download=True,      # télécharge si non présent
    transform=ToTensor()  # transformation des images en tenseurs
)

# Charger le dataset de test de la même façon
test_data = datasets.FashionMNIST(
    root='data/',
    train=False,        # jeu de test
    download=True,
    transform=ToTensor()
)

print(f"Train set size: {len(train_data)}")
print(f"Test set size: {len(test_data)}")

## 2) Model: a tiny autoencoder
We’ll encode 28×28 images → 2D latent, then decode back to 28×28.

**Exercise 2.1** — Implement the following architecture:
- Encoder: Linear(784→256) → Tanh → Linear(256→2)
- Decoder: Linear(2→256) → Tanh → Linear(256→512) → ReLU → Linear(512→784)

Return the output reshaped to the original image shape.


In [ ]:
# TODO: Exercise 2.1 — implement the autoencoder
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.encoder = nn.Sequential(
            nn.Linear(784, 256),
            nn.Tanh(),
            nn.Linear(256, 2)
        )
        self.decoder = nn.Sequential(
            nn.Linear(2, 256),
            nn.Tanh(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 784)
        )
        
    def forward(self, x):
        initial_shape = x.shape  # e.g. (batch_size, 28, 28)
        x = self.flatten(x)      # aplatir en (batch_size, 784)
        x = self.encoder(x)      # encoder vers un vecteur latent 2D
        x = self.decoder(x)      # decoder vers un vecteur de taille 784
        return x.view(initial_shape)  # reshape en (batch_size, 28, 28)

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): Tanh()
    (2): Linear(in_features=256, out_features=2, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=2, out_features=256, bias=True)
    (1): Tanh()
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=784, bias=True)
  )
)


## 3) Loss and Optimizer
**Exercise 3.1** — Use MSE loss to measure reconstruction error. Choose Adam with learning rate 1e-3.


In [ ]:
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

## 4) Training & Evaluation loops
**Exercise 4.1** — Implement a standard training loop.


In [ ]:
def train(dataloader, model, loss_fn, optimizer, device):
    model.train()
    total_loss = 0
    for batch, (X, _) in enumerate(dataloader):
        X = X.to(device)
        
        # Forward pass : reconstruction
        reconstruction = model(X)
        
        # Calcul de la perte
        loss = loss_fn(reconstruction, X)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    avg_loss = total_loss / len(dataloader)
    print(f"Training loss: {avg_loss:.4f}")
    return avg_loss

**Exercise 4.2** — Implement a simple test loop computing average reconstruction loss.


In [ ]:
def test(dataloader, model, loss_fn, device):
    model.eval()  # mode évaluation, désactivation du dropout, batchnorm, etc.
    total_loss = 0
    
    with torch.no_grad():  # pas de calcul de gradients
        for X, _ in dataloader:
            X = X.to(device)
            reconstruction = model(X)
            loss = loss_fn(reconstruction, X)
            total_loss += loss.item()
    
    avg_loss = total_loss / len(dataloader)
    print(f"Test loss: {avg_loss:.4f}")
    return avg_loss

**Exercise 4.3** — Train for a few epochs (e.g., 5) and observe the printed losses.

In [ ]:
num_epochs = 5

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    
    train_loss = train(train_dataloader, model, loss_function, optimizer, device)
    test_loss = test(test_dataloader, model, loss_function, device)
    
    print(f"Epoch {epoch+1} - Train loss: {train_loss:.4f}, Test loss: {test_loss:.4f}\n")

## 5) Qualitative check — show input vs reconstruction
**Exercise 5.1** — Plot a few inputs and their reconstructions side-by-side.

Hint: Turn off gradients with `torch.no_grad()` and move tensors to CPU for plotting.